## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-05-22-51-18 +0000,nypost,Chicago police ordered to ignore border patrol...,https://nypost.com/2025/10/05/us-news/chicago-...
1,2025-10-05-22-50-20 +0000,bbc,Trump authorises deployment of 300 National Gu...,https://www.bbc.com/news/articles/c2dnk0ee6pyo...
2,2025-10-05-22-48-08 +0000,nypost,Free ride over for electric vehicles on the Lo...,https://nypost.com/2025/10/05/us-news/free-rid...
3,2025-10-05-22-47-29 +0000,nyt,Mother Is Charged in Fatal Shooting of Two of ...,https://www.nytimes.com/2025/10/04/us/texas-sh...
4,2025-10-05-22-40-41 +0000,nypost,Freed American hostage Keith Siegel says Hamas...,https://nypost.com/2025/10/05/us-news/freed-am...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2316/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
13,trump,36
61,gaza,11
0,chicago,10
16,national,10
17,guard,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
101,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,133
34,2025-10-05-20-28-02 +0000,nypost,"Trump defies federal court, sends 300 Californ...",https://nypost.com/2025/10/05/us-news/trump-de...,93
80,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...,89
5,2025-10-05-22-37-00 +0000,wsj,Trump Sends California Troops to Portland Afte...,https://www.wsj.com/politics/policy/california...,79
28,2025-10-05-20-56-35 +0000,nyt,Trump Sends California National Guard Troops t...,https://www.nytimes.com/2025/10/05/us/trump-na...,79


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
101,133,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
181,39,2025-10-04-23-05-09 +0000,nypost,Youngkin calls for embattled Dem AG hopeful Ja...,https://nypost.com/2025/10/04/us-news/virginia...
0,35,2025-10-05-22-51-18 +0000,nypost,Chicago police ordered to ignore border patrol...,https://nypost.com/2025/10/05/us-news/chicago-...
78,34,2025-10-05-17-52-57 +0000,bbc,Trump says he thinks Gaza hostages will be fre...,https://www.bbc.com/news/articles/cj3y6g43248o...
26,26,2025-10-05-20-59-08 +0000,nypost,Jersey City Mayor Steve Fulop being eyed for b...,https://nypost.com/2025/10/05/us-news/jersey-c...
135,26,2025-10-05-09-30-00 +0000,wsj,"As Laura Loomer, an influential loyalist to Pr...",https://www.wsj.com/politics/policy/laura-loom...
23,25,2025-10-05-21-24-00 +0000,wsj,The federal government shutdown is dragging to...,https://www.wsj.com/politics/policy/government...
180,25,2025-10-04-23-14-00 +0000,wsj,Defense Secretary Pete Hegseth’s emphasis on p...,https://www.wsj.com/politics/national-security...
14,24,2025-10-05-21-57-06 +0000,startribune,Neal: War of words between Napheesa Collier an...,https://www.startribune.com/napheesa-collier-c...
18,24,2025-10-05-21-33-27 +0000,nypost,NJ high school teacher charged with sexually a...,https://nypost.com/2025/10/05/us-news/nj-high-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
